In [3]:
# 추가 수정
import pandas as pd
import os
import numpy as np
from datetime import datetime, timedelta

In [4]:
# 1단계: 법정동 데이터 전처리
# 1-1 법정동 데이터 로드하기
file_path = "data/법정동코드 조회자료.xls"
dong_df = pd.read_excel(file_path)
print(f"✓ 법정동 데이터 로드 완료: {dong_df.shape}")

✓ 법정동 데이터 로드 완료: (1112, 3)


In [5]:
# 1-2 전체코드 분리 (앞 5자리/뒤 5자리)
dong_df["법정동코드"] = dong_df["법정동코드"].astype(str).str.zfill(10)
dong_df["시군구코드"] = dong_df["법정동코드"].str[:5]
dong_df["법정동코드_하위"] = dong_df["법정동코드"].str[5:]


In [6]:
# 1-3 상위 행정구역 제거
before = len(dong_df)
dong_df = dong_df[dong_df["법정동코드_하위"] != "00000"].copy()
print(f"✓ 상위 행정구역 제거: {before} → {len(dong_df)}")


✓ 상위 행정구역 제거: 1112 → 1086


In [7]:
#1-4 법정동 전체명 분리
dong_df["법정동명"] = dong_df["법정동명"].astype(str).str.strip()
dong_df["시군구명"] = dong_df["법정동명"].apply(lambda x: " ".join(x.split()[:-1]))
dong_df["법정동명_하위"] = dong_df["법정동명"].apply(lambda x: x.split()[-1])

dong_clean = dong_df[[
    "법정동코드", "시군구코드", "법정동코드_하위", 
    "시군구명", "법정동명_하위"
]].drop_duplicates().reset_index(drop=True)

dong_clean.to_csv("data/법정동_마스터.csv", index=False, encoding="utf-8-sig")
print(f"✓ 법정동 마스터 생성: {dong_clean.shape}\n")

✓ 법정동 마스터 생성: (1086, 5)



In [8]:
# 2단계: 전력량 데이터 전처리 (고도화)
# 2-1 전력량 데이터(법정동별시간별전력사용량.csv) 로드하기
power_path = "data/법정동별시간별전력사용량.csv"
try:
    power_df = pd.read_csv(power_path, encoding="utf-8-sig")
except UnicodeDecodeError:
    power_df = pd.read_csv(power_path, encoding="cp949")
print(f"✓ 전력량 데이터 로드: {power_df.shape}")

✓ 전력량 데이터 로드: (9754804, 5)


In [9]:
# 2-2 중복 제거
before = len(power_df)
power_df = power_df.drop_duplicates().copy()
print(f"✓ 중복 제거: {before} → {len(power_df)}")

✓ 중복 제거: 9754804 → 4348560


In [10]:
# 법정동코드 생성
power_df["SIGUNGU_CD"] = power_df["SIGUNGU_CD"].astype(str).str.zfill(5)
power_df["BJDONG_CD"] = power_df["BJDONG_CD"].astype(str).str.zfill(5)
power_df["법정동코드"] = power_df["SIGUNGU_CD"] + power_df["BJDONG_CD"]

# 일시 생성
power_df["USE_YM"] = power_df["USE_YM"].astype(str)
power_df["USE_HM"] = power_df["USE_HM"].astype(str).str.zfill(4)
power_df["일시"] = pd.to_datetime(
    power_df["USE_YM"].str[:8] + power_df["USE_HM"],
    format="%Y%m%d%H%M", 
    errors="coerce"
)
power_df["일자"] = power_df["일시"].dt.date

In [20]:
# 2-3 전력데이터를 시간당 -> 일당 변경해보기
# 전력량 숫자 변환
power_df["FDRCT_VLD_KWH"] = pd.to_numeric(power_df["FDRCT_VLD_KWH"], errors="coerce")

# 🔥 NEW: 시간 누락/중복 검증
print("\n[시간 정합성 검증]")
time_check = power_df.groupby(["법정동코드", "일시"]).size().reset_index(name='count')
duplicates = time_check[time_check['count'] > 1]
if len(duplicates) > 0:
    print(f"⚠️ 중복 시간 발견: {len(duplicates)}건 → 평균값으로 집계")
    power_df = power_df.groupby(["법정동코드", "일시"], as_index=False)["FDRCT_VLD_KWH"].mean()
else:
    print("✓ 중복 시간 없음")

# 🔥 NEW: 시간별 데이터로 변환 (일별 대신)
power_hourly = power_df.copy()
power_hourly["일자"] = power_hourly["일시"].dt.date
power_hourly["시간"] = power_hourly["일시"].dt.hour
power_hourly["년"] = power_hourly["일시"].dt.year  
power_hourly["월"] = power_hourly["일시"].dt.month

print(f"✓ 시간별 데이터 생성: {power_hourly.shape}")




[시간 정합성 검증]
✓ 중복 시간 없음
✓ 시간별 데이터 생성: (4348560, 11)


In [21]:
# 🔥 NEW: 이상치 완화 (제거 X, Winsorizing)
print("\n[전력량 이상치 완화]")
def winsorize_power(group):
    upper = group["FDRCT_VLD_KWH"].quantile(0.995)
    lower = group["FDRCT_VLD_KWH"].quantile(0.005)
    group["전력량_원본"] = group["FDRCT_VLD_KWH"]
    group["전력량_완화"] = group["FDRCT_VLD_KWH"].clip(lower, upper)
    return group

power_hourly = power_hourly.groupby("법정동코드", group_keys=False).apply(winsorize_power)
print(f"✓ Winsorizing 완료 (0.5~99.5% 기준)")

# 🔥 NEW: 저전력 플래그 변수
power_hourly["저전력_플래그"] = 0
for dong in power_hourly["법정동코드"].unique():
    mask = power_hourly["법정동코드"] == dong
    threshold = power_hourly.loc[mask, "전력량_완화"].quantile(0.02)
    power_hourly.loc[mask & (power_hourly["전력량_완화"] < threshold), "저전력_플래그"] = 1

print(f"✓ 저전력 플래그 생성 (하위 2% 기준)")



[전력량 이상치 완화]
✓ Winsorizing 완료 (0.5~99.5% 기준)
✓ 저전력 플래그 생성 (하위 2% 기준)


In [15]:
# 2-4. 법정동 병합
merged_power = power_hourly.merge(dong_clean, on="법정동코드", how="left")
print(f"✓ 법정동 마스터와 병합: {merged_power.shape}")

merged_power.to_csv("data/전력_시간별_병합.csv", index=False, encoding="utf-8-sig")
print("✓ 저장: data/전력_시간별_병합.csv\n")


✓ 법정동 마스터와 병합: (4348560, 16)
✓ 저장: data/전력_시간별_병합.csv



In [16]:
# 3단계: 날씨 데이터 전처리 (고도화)
 # 1-1 날씨데이터의 빈측치나 결측치 알아보기
base_dir = "data/weather_data"

# 시간별(HR) 데이터 로드 (일별 대신 시간별 사용)
hr_files = [
    "SURFACE_ASOS_108_HR_2022_2022_2023.csv",
    "SURFACE_ASOS_108_HR_2023_2023_2024.csv",
    "SURFACE_ASOS_108_HR_2024_2024_2025.csv"
]

hr_dfs = []
for f in hr_files:
    fpath = os.path.join(base_dir, f)
    try:
        df = pd.read_csv(fpath, encoding="utf-8-sig")
    except UnicodeDecodeError:
        df = pd.read_csv(fpath, encoding="cp949")
    hr_dfs.append(df)

weather_hr = pd.concat(hr_dfs, ignore_index=True)
print(f"✓ 시간별 날씨 데이터 로드: {weather_hr.shape}")


✓ 시간별 날씨 데이터 로드: (26304, 27)


In [17]:
# 날짜/시간 컬럼 탐지
date_col = None
for c in weather_hr.columns:
    if any(k in str(c).lower() for k in ["tm", "일시", "time"]):
        s = pd.to_datetime(weather_hr[c], errors="coerce")
        if s.notna().sum() > len(weather_hr) * 0.9:
            date_col = c
            weather_hr["일시"] = s
            break

if date_col is None:
    raise ValueError("날씨 데이터에서 날짜 컬럼을 찾을 수 없습니다.")

weather_hr["일자"] = weather_hr["일시"].dt.date
weather_hr["시간"] = weather_hr["일시"].dt.hour
print(f"✓ 날짜/시간 변환 완료: {date_col} → 일시")

# 필요한 컬럼 매핑 (실제 존재하는 컬럼명 사용)
col_mapping = {
    "기온(°C)": "기온",
    "강수량(mm)": "강수량", 
    "풍속(m/s)": "풍속",
    "습도(%)": "습도"
}

use_cols = ["일시", "일자", "시간"]
for orig, new in col_mapping.items():
    if orig in weather_hr.columns:
        weather_hr[new] = pd.to_numeric(weather_hr[orig], errors="coerce")
        use_cols.append(new)

weather_clean = weather_hr[use_cols].copy()
print(f"✓ 날씨 컬럼 선택: {weather_clean.columns.tolist()}")


✓ 날짜/시간 변환 완료: 일시 → 일시
✓ 날씨 컬럼 선택: ['일시', '일자', '시간', '기온', '강수량', '풍속', '습도']


In [18]:
# 🔥 NEW: 결측치 처리 (강수량=0, 나머지=보간)
print("\n[결측치 처리]")
if "강수량" in weather_clean.columns:
    weather_clean["강수량"] = weather_clean["강수량"].fillna(0)
    print("✓ 강수량 결측치 → 0")

for col in ["기온", "풍속", "습도"]:
    if col in weather_clean.columns:
        before_na = weather_clean[col].isna().sum()
        weather_clean[col] = weather_clean[col].interpolate(method="linear")
        weather_clean[col] = weather_clean[col].fillna(method="ffill").fillna(method="bfill")
        print(f"✓ {col} 결측치 보간: {before_na} → {weather_clean[col].isna().sum()}")

# 🔥 NEW: 체감 관련 파생 변수
print("\n[기상 파생 변수 생성]")

if "기온" in weather_clean.columns and "습도" in weather_clean.columns:
    # 불쾌지수
    T = weather_clean["기온"]
    RH = weather_clean["습도"]
    weather_clean["불쾌지수"] = 0.81 * T + 0.01 * RH * (0.99 * T - 14.3) + 46.3
    print("✓ 불쾌지수 생성")
    
    # 냉난방도일
    weather_clean["냉방도일"] = (weather_clean["기온"] - 24).clip(lower=0)
    weather_clean["난방도일"] = (18 - weather_clean["기온"]).clip(lower=0)
    print("✓ 냉난방도일 생성")

# 🔥 NEW: 기온 변화량
if "기온" in weather_clean.columns:
    weather_clean = weather_clean.sort_values(["일시"]).reset_index(drop=True)
    weather_clean["기온변화_1h"] = weather_clean["기온"].diff(1)
    weather_clean["기온변화_24h"] = weather_clean["기온"].diff(24)
    weather_clean["기온변화_1h"] = weather_clean["기온변화_1h"].fillna(0)
    weather_clean["기온변화_24h"] = weather_clean["기온변화_24h"].fillna(0)
    print("✓ 기온 변화량 생성 (1h, 24h)")

# 🔥 NEW: 강수 여부 플래그
if "강수량" in weather_clean.columns:
    weather_clean["강수여부"] = (weather_clean["강수량"] > 0).astype(int)
    print("✓ 강수 여부 플래그 생성")

# 시군구코드 부여 (서울: 11000)
weather_clean["시군구코드"] = "11000"

weather_clean.to_csv("data/날씨_시간별_정제.csv", index=False, encoding="utf-8-sig")
print(f"✓ 저장: data/날씨_시간별_정제.csv\n")


[결측치 처리]
✓ 강수량 결측치 → 0
✓ 기온 결측치 보간: 0 → 0
✓ 풍속 결측치 보간: 68 → 0
✓ 습도 결측치 보간: 0 → 0

[기상 파생 변수 생성]
✓ 불쾌지수 생성
✓ 냉난방도일 생성
✓ 기온 변화량 생성 (1h, 24h)
✓ 강수 여부 플래그 생성
✓ 저장: data/날씨_시간별_정제.csv



In [19]:
# 3-4 다른 데이터들과 합치기
# 일시 기준으로 병합 (정확한 시간 매칭)
final_df = merged_power.merge(
    weather_clean,
    on=["일자", "시간", "시군구코드"],
    how="left",
    suffixes=("", "_날씨")
)

print(f"✓ 전력+날씨 병합 완료: {final_df.shape}")

# 🔥 NEW: 시간 파생 변수
print("\n[시간 파생 변수 생성]")
final_df["일시"] = pd.to_datetime(final_df["일시"])
final_df["년"] = final_df["일시"].dt.year
final_df["월"] = final_df["일시"].dt.month
final_df["일"] = final_df["일시"].dt.day
final_df["요일"] = final_df["일시"].dt.dayofweek  # 0=월요일
final_df["주말여부"] = (final_df["요일"] >= 5).astype(int)

# 🔥 NEW: 공휴일 플래그 (간단 버전 - 실제론 공공데이터 API 사용 권장)
holidays_2022_2025 = [
    # 2022년
    "2022-01-01", "2022-01-31", "2022-02-01", "2022-02-02", "2022-03-01",
    "2022-03-09", "2022-05-05", "2022-05-08", "2022-06-06", "2022-08-15",
    "2022-09-09", "2022-09-10", "2022-09-11", "2022-10-03", "2022-10-09",
    "2022-10-10", "2022-12-25",
    # 2023년
    "2023-01-01", "2023-01-21", "2023-01-22", "2023-01-23", "2023-01-24",
    "2023-03-01", "2023-05-05", "2023-05-27", "2023-06-06", "2023-08-15",
    "2023-09-28", "2023-09-29", "2023-09-30", "2023-10-03", "2023-10-09",
    "2023-12-25",
    # 2024년
    "2024-01-01", "2024-02-09", "2024-02-10", "2024-02-11", "2024-02-12",
    "2024-03-01", "2024-04-10", "2024-05-05", "2024-05-06", "2024-05-15",
    "2024-06-06", "2024-08-15", "2024-09-16", "2024-09-17", "2024-09-18",
    "2024-10-03", "2024-10-09", "2024-12-25",
    # 2025년
    "2025-01-01", "2025-01-28", "2025-01-29", "2025-01-30", "2025-03-01",
    "2025-03-03", "2025-05-05", "2025-05-06", "2025-06-06", "2025-08-15",
    "2025-10-03", "2025-10-05", "2025-10-06", "2025-10-07", "2025-10-08",
    "2025-10-09", "2025-12-25"
]
holidays = pd.to_datetime(holidays_2022_2025).date
final_df["공휴일여부"] = final_df["일자"].isin(holidays).astype(int)

print("✓ 시간 파생 변수 생성 (년월일, 요일, 주말, 공휴일)")

# 🔥 NEW: 법정동별 통계 Feature (Train 기간 기준)
print("\n[법정동별 통계 Feature 생성 (Train 기간 기준)]")

# Train 기간: 2022~2024년
train_mask = final_df["년"] <= 2024
train_data = final_df[train_mask].copy()

dong_stats = train_data.groupby("법정동코드").agg({
    "전력량_완화": ["mean", "std", "max"]
}).reset_index()
dong_stats.columns = ["법정동코드", "법정동_평균전력", "법정동_표준편차", "법정동_최대전력"]

# 시간대별 평균
dong_hour_stats = train_data.groupby(["법정동코드", "시간"])["전력량_완화"].mean().reset_index()
dong_hour_stats.columns = ["법정동코드", "시간", "법정동_시간평균전력"]

# 병합
final_df = final_df.merge(dong_stats, on="법정동코드", how="left")
final_df = final_df.merge(dong_hour_stats, on=["법정동코드", "시간"], how="left")

print("✓ 법정동 통계 Feature 병합 완료")

# 최종 저장
final_df.to_csv("data/최종_통합데이터_시간별.csv", index=False, encoding="utf-8-sig")
print(f"\n✅ 최종 데이터 저장 완료: data/최종_통합데이터_시간별.csv")
print(f"   - 총 데이터 수: {len(final_df):,}개")
print(f"   - 컬럼 수: {len(final_df.columns)}개")
print(f"   - 기간: {final_df['일자'].min()} ~ {final_df['일자'].max()}")
print(f"   - 법정동 수: {final_df['법정동코드'].nunique()}개")

# 데이터 품질 체크
print("\n" + "=" * 60)
print("데이터 품질 체크")
print("=" * 60)
print("결측치 현황:")
null_summary = final_df.isnull().sum()
print(null_summary[null_summary > 0])

print("\n주요 컬럼 통계:")
important_cols = ["전력량_완화", "기온", "불쾌지수", "냉방도일", "난방도일"]
for col in important_cols:
    if col in final_df.columns:
        print(f"\n{col}:")
        print(final_df[col].describe())

print("\n✅ 전체 전처리 완료!")

✓ 전력+날씨 병합 완료: (4348560, 27)

[시간 파생 변수 생성]
✓ 시간 파생 변수 생성 (년월일, 요일, 주말, 공휴일)

[법정동별 통계 Feature 생성 (Train 기간 기준)]
✓ 법정동 통계 Feature 병합 완료

✅ 최종 데이터 저장 완료: data/최종_통합데이터_시간별.csv
   - 총 데이터 수: 4,348,560개
   - 컬럼 수: 37개


TypeError: '<=' not supported between instances of 'datetime.date' and 'float'